# Sonde3 
##  Reads and converts binary water quality environmental instrument data to a DataFrame


### I.  Example Usage

Lets dive in!  

We have a example water quality instrument binary file `"tests/ysi_test_files/SA08.dat"`.  This file was generated by a YSI 600LS instrument and is in proprietary binary format.

#### Using the `sonde()` function we:

1.  `autodetect()` the file type and pass to the correct parser function 
2.  `read_ysi()` the binary file and convert to pandas DataFrame
3.  Transform all datetimes to the UTC timezone
4.  Standardize the units to metric and rename the columns to standard name conventions
3.  Pass the DataFrame to `calculate_salinity_psu()` and `calculate_do_mgl()` to apply standard formulas to generate the salinity and dissolved oxygen columns.

In [1]:
import sonde3
import pandas   
metadata, df = sonde3.sonde("tests/ysi_test_files/SA08.dat", remove_invalids=False)


C:\Users\ETurner\Desktop\python\sonde3\sonde3\sonde.py:19: UserWarning: Info: No time zone was set for file, assuming records are recorded in CST
  metadata, df = formats.read_ysi(filename, tzinfo)
C:\Users\ETurner\AppData\Local\Continuum\Anaconda3\lib\site-packages\seawater\library.py:363: RuntimeWarning: invalid value encountered in sqrt
  Rtx = (rt) ** 0.5


#### Why the runtime warnings?

1.  The YSI instrument files don't contain any timezone information.  Therefore, the function has to assume that the timezone of the file to make the UTC conversion.

2. Often raw instrument files will contain impossible & incorrect values in the beginning and end of the file.  Examples: negative values for salinity or dissolved oxygen percentage.  `sonde3` does not trim the raw file, or perform QA analysis.  `sonde3` will pass the values as they were recorded by the instrument.

To automatically convert invalid negative values to zero, either set the flag **remove_invalids=False** or remove it altogether, as this is the default behavior of the package

##### We can now interact with the two dataframes produced by `sonde3`:


In [2]:
df.info() 
df.head() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 9 columns):
Datetime_(UTC)                       700 non-null datetime64[ns, UTC]
water_temp_c                         700 non-null float64
water_conductivity_mS/cm             700 non-null float64
water_depth_m_nonvented              700 non-null float64
water_DO_%                           700 non-null float64
instrument_battery_voltage           700 non-null float64
water_specific_conductivity_mS/cm    700 non-null float64
water_salinity_PSU                   678 non-null float64
water_DO_mgl                         678 non-null float64
dtypes: datetime64[ns, UTC](1), float64(8)
memory usage: 49.3 KB


,Datetime_(UTC),water_temp_c,water_conductivity_mS/cm,water_depth_m_nonvented,water_DO_%,instrument_battery_voltage,water_specific_conductivity_mS/cm,water_salinity_PSU,water_DO_mgl
0,2008-07-16 12:00:31+00:00,28.998718,0.000037,0.010862,93.391418,6.09375,0.000035,0.013536,7.183420
1,2008-07-16 13:00:31+00:00,28.482361,0.000059,0.016358,96.765137,6.09375,0.000055,0.013326,7.510631
2,2008-07-16 14:00:31+00:00,27.257385,0.000546,0.017263,103.529358,6.09375,0.000524,0.012655,8.212117
3,2008-07-16 15:00:31+00:00,29.507751,21.301758,0.542648,93.055725,6.09375,19.613107,11.601472,6.655432
4,2008-07-16 16:00:31+00:00,29.762268,21.454102,0.557098,94.188690,6.09375,19.665354,11.631321,6.706414


In [3]:
metadata

,Model,Manufacturer,Instrument_Serial_Number,Station,Deployment_Setup_Time,Filename,Deployment_Start_Time,Deployment_Stop_Time
0,600,YSI,1012,SANT_CDT,,SA08.dat,2008-07-16 12:00:31+00:00,2008-08-14 15:00:16+00:00


### II.  Working with time zones


What if data was collected outside of US/Central time?  Pass the timezone information to `sonde3.sonde`:

In [4]:
import pytz
metadata, df = sonde3.sonde("tests/ysi_test_files/SA08.dat", pytz.timezone('US/Eastern'))
df.head()

,Datetime_(UTC),water_temp_c,water_conductivity_mS/cm,water_depth_m_nonvented,water_DO_%,instrument_battery_voltage,water_specific_conductivity_mS/cm,water_salinity_PSU,water_DO_mgl
0,2008-07-16 11:00:31+00:00,28.998718,0.000037,0.010862,93.391418,6.09375,0.000035,0.013536,7.183420
1,2008-07-16 12:00:31+00:00,28.482361,0.000059,0.016358,96.765137,6.09375,0.000055,0.013326,7.510631
2,2008-07-16 13:00:31+00:00,27.257385,0.000546,0.017263,103.529358,6.09375,0.000524,0.012655,8.212117
3,2008-07-16 14:00:31+00:00,29.507751,21.301758,0.542648,93.055725,6.09375,19.613107,11.601472,6.655432
4,2008-07-16 15:00:31+00:00,29.762268,21.454102,0.557098,94.188690,6.09375,19.665354,11.631321,6.706414


### III. Autodetecting files



Curious about what kind of instrument files you have in a directory?  Apply the `sonde3.autodetect` method:

```python
#what kind of file is this??
sonde3.autodetect("tests/greenspan_test_files/RIOA_20060718_CDT_GS7837.xls") 
```

In [5]:
sonde3.autodetect("tests/greenspan_test_files/RIOA_20060718_CDT_GS7837.xls") 

'greenspan_xls'

Copy this code snippet to the notebook line below to run `autodetect()` on all of the use-case examples in the `sonde3` package:

```python
#this script runs through all of the text examples and prints out the autodetect results
import os

root_dir = 'tests'
results = []
for directory, subdirectories, files in os.walk(root_dir):
    for file in files:
        if "_test.txt" in file:
            continue
        os.path.join(directory, file)
        results.append(os.path.join(directory, file) + ' ' + sonde3.autodetect(os.path.join(directory, file)))

results 
```

### IV. Generating Salinity and Dissolved Oxygen

Typically deployed water quality instruments do not compute all rows of data internally.  Instead, these are calculated by the program used to read the file back at the lab.  For example, YSI instruments do not compute salinity or dissolved oxygen concentration.  This can cause confusion because when viewing a YSI *\*.dat* file in YSI's ECOWIN, or Ecowatch Lite program displays salinity and DO mg/L.  However, the raw binary file does not include these rows as they were not physically collected by the instrument during deployment.

For example, lets read the raw binary file of the example file `"tests/ysi_test_files/SA08.dat"` and see what it contains:

In [6]:
metadata, SA08_BIN = sonde3.read_ysi("tests/ysi_test_files/SA08.dat",pytz.timezone('US/Central'))
SA08_BIN.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 6 columns):
Datetime_(UTC)                700 non-null datetime64[ns, UTC]
water_temp_c                  700 non-null float64
water_conductivity_mS/cm      700 non-null float64
water_depth_m_nonvented       700 non-null float64
water_DO_%                    700 non-null float64
instrument_battery_voltage    700 non-null float64
dtypes: datetime64[ns, UTC](1), float64(5)
memory usage: 32.9 KB


The pandas DataFrame contains the columns for conductivity and % DO saturation, but not for salinity and DO concentration.

For comparision, lets read the comma separated version of this file that was produced by the proprietary YSI Ecowin program:

In [7]:
metadata, SA08_CSV = sonde3.read_ysi_ascii("tests/ysi_test_files/SA08.CDF", pytz.timezone('US/Central'),delim=",")


The ECOWIN exported comma separated file has far more columns! The extra columns were derived through formulas using the instrumnet observed measurments.

To generate these columns pass the SA08_BIN DataFrame to calculate Salinity (PSU) and Dissolved Oxygen (mg/L) using `sonde3.calculate_salinity_psu()` and `sonde3.calculate_do_mgl()`

We can then compare our computed results to the ECOwatch program results:

In [8]:
SA08_BIN = sonde3.calculate_salinity_psu(SA08_BIN)
SA08_BIN = sonde3.calculate_do_mgl(SA08_BIN)
SA08_BIN.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 8 columns):
Datetime_(UTC)                700 non-null datetime64[ns, UTC]
water_temp_c                  700 non-null float64
water_conductivity_mS/cm      700 non-null float64
water_depth_m_nonvented       700 non-null float64
water_DO_%                    700 non-null float64
instrument_battery_voltage    700 non-null float64
water_salinity_PSU            678 non-null float64
water_DO_mgl                  678 non-null float64
dtypes: datetime64[ns, UTC](1), float64(7)
memory usage: 43.8 KB


C:\Users\ETurner\AppData\Local\Continuum\Anaconda3\lib\site-packages\seawater\library.py:363: RuntimeWarning: invalid value encountered in sqrt
  Rtx = (rt) ** 0.5


So, why does `sonde3` produce less values for salinity and DO than the comma separated file?  Is this a bug?

*NO!*  If you recal from the `sqrt()` warning above, some of the values in the file are invalid.  This produces null (**NaN**) values in those rows.

In [9]:
SA08_BIN.tail()

,Datetime_(UTC),water_temp_c,water_conductivity_mS/cm,water_depth_m_nonvented,water_DO_%,instrument_battery_voltage,water_salinity_PSU,water_DO_mgl
695,2008-08-14 11:00:16+00:00,23.977051,-0.000006,-0.052356,-0.006866,5.546875,NaN,NaN
696,2008-08-14 12:00:16+00:00,23.938599,-0.000005,-0.050262,-0.006866,5.468750,NaN,NaN
697,2008-08-14 13:00:16+00:00,24.039307,-0.000005,-0.045875,-0.006866,5.468750,NaN,NaN
698,2008-08-14 14:00:16+00:00,24.107056,-0.000005,-0.038671,-0.006866,5.468750,NaN,NaN
699,2008-08-14 15:00:16+00:00,24.177551,-0.000006,-0.034785,-0.006866,5.468750,NaN,NaN


Lets check our two files to see if our conversion methods are those used by YSI.  

Lets use `numpy.random` to pick a row to check:



In [10]:
from numpy import random
row = random.randint(2,677) # pick random row

sub1 =  SA08_BIN.iloc[row:row+1]  #row one binary file
sub2 = SA08_CSV.iloc[row:row+1]   #row two csv file

pandas.concat([sub1, sub2], axis=0,join="inner")

,Datetime_(UTC),water_temp_c,water_conductivity_mS/cm,water_depth_m_nonvented,water_DO_%,instrument_battery_voltage,water_salinity_PSU,water_DO_mgl
628,2008-08-11 16:00:31+00:00,28.985901,31.544922,0.541229,70.027161,5.703125,18.026141,4.876619
628,2008-08-11 16:00:31+00:00,28.990000,31.545000,0.541000,70.000000,5.700000,18.030000,4.880000


Looks the same!  Notice, however, that the csv file values are rounded due to the fixed digit precision.

### V. Files from other Manufacturers
#### a. Hydrotech

In [11]:
metadata, mydf = sonde3.sonde("tests/hydrotech_test_files/0109DELT.CSV", pytz.timezone('US/Central'))

In [12]:
metadata

,Manufacturer,Instrument_Serial_Number,Model,Station,Deployment_Setup_Time,Deployment_Start_Time,Deployment_Stop_Time,Filename
0,Hydrotech,081107-D,MiniSonde4a,0109delt,2009-01-13 06:00:00+00:00,2009-01-14 13:00:00+00:00,2009-02-12 16:00:00+00:00,0109DELT.CSV


In [13]:
mydf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 700 entries, 13 to 712
Data columns (total 7 columns):
Datetime_(UTC)                       700 non-null datetime64[ns, UTC]
water_temp_c                         700 non-null float64
water_specific_conductivity_mS/cm    700 non-null float64
water_salinity_PSU                   700 non-null float64
water_depth_m_nonvented              700 non-null float64
instrument_battery_voltage           700 non-null float64
water_conductivity_mS/cm             700 non-null float64
dtypes: datetime64[ns, UTC](1), float64(6)
memory usage: 43.8 KB


In [14]:
mydf.head()

,Datetime_(UTC),water_temp_c,water_specific_conductivity_mS/cm,water_salinity_PSU,water_depth_m_nonvented,instrument_battery_voltage,water_conductivity_mS/cm
13,2009-01-14 13:00:00+00:00,12.81,0.866,0.428305,-0.0,6.0,0.664370
14,2009-01-14 14:00:00+00:00,12.57,0.868,0.429304,-0.0,6.0,0.661926
15,2009-01-14 15:00:00+00:00,12.34,0.867,0.428759,-0.0,6.0,0.657354
16,2009-01-14 16:00:00+00:00,12.59,0.868,0.429306,-0.0,6.0,0.662257
17,2009-01-14 17:00:00+00:00,11.78,0.883,0.436875,-0.0,6.0,0.660041


In [15]:
mydf.tail()

,Datetime_(UTC),water_temp_c,water_specific_conductivity_mS/cm,water_salinity_PSU,water_depth_m_nonvented,instrument_battery_voltage,water_conductivity_mS/cm
708,2009-02-12 12:00:00+00:00,19.24,34.9,22.008874,1.03,3.9,31.060442
709,2009-02-12 13:00:00+00:00,19.22,34.9,22.008973,1.02,3.9,31.047110
710,2009-02-12 14:00:00+00:00,19.31,35.1,22.147645,1.02,3.9,31.285367
711,2009-02-12 15:00:00+00:00,19.44,35.8,22.634720,1.02,3.9,31.998183
712,2009-02-12 16:00:00+00:00,19.34,34.9,22.008374,1.03,0.0,31.127101


### VI. Package Validation Tests

#### A.  Check to see if the package is correctly handling daylight savings time.

The test case dataset **tests/ysi_test_files/0108BAYT.csv** was collected during a daylight savings transition.  These three files were processed differently at the time of instrument processing.  The *.dat* file is the raw binary file from the instrument.  The *CDF* file was produced by the Ecowatch program.  The *cvs* was produced by converting the *CDF* file in ms excel to a comma separated file.  If the times were not consistant between the three files it would indicate **sonde3** is not processing datetime correctly.

In [16]:
#import all three versions of the file ,CDF, csv, and binary *.dat
metadata, baytcsv = sonde3.sonde("tests/ysi_test_files/0108BAYT.csv", pytz.timezone('US/Central'))
metadata, baytdat = sonde3.sonde("tests/ysi_test_files/0108BAYT.dat", pytz.timezone('US/Central'))
metadata, baytCDF = sonde3.sonde("tests/ysi_test_files/0108BAYT.CDF", pytz.timezone('US/Central'))

In [17]:
#cut the first & last row from each file.
pandas.concat([baytCDF.iloc[0:1], baytdat.iloc[0:1], baytcsv.iloc[0:1] , \
               baytCDF.iloc[-1:], baytdat.iloc[-1:], baytcsv.iloc[-1:]], axis=0,join="inner")

,Datetime_(UTC),water_temp_c,water_specific_conductivity_mS/cm,water_depth_m_nonvented,water_DO_%,water_conductivity_mS/cm,water_DO_mgl,water_salinity_PSU
0,2008-01-29 15:00:33+00:00,22.560000,-0.0,-0.0,103.200000,-0.0,8.927547,0.011370
0,2008-01-29 15:00:33+00:00,22.562561,-0.0,-0.0,103.227234,-0.0,8.929467,0.011371
0,2008-01-29 15:00:00+00:00,22.560000,0.0,-0.0,103.200000,0.0,8.927547,0.011370
1061,2008-03-13 19:00:33+00:00,22.510000,-0.0,-0.0,104.300000,-0.0,9.031326,0.011350
1061,2008-03-13 19:00:33+00:00,22.510376,-0.0,-0.0,104.284668,-0.0,9.029934,0.011350
1061,2008-03-13 19:00:00+00:00,22.510000,0.0,-0.0,104.300000,0.0,9.031326,0.011350


In [18]:
import os
def save_data(path, file, df):
      if (df.to_csv(path + file + '.csv', index = False)):
          return True
      else:
          return False

In [19]:
save_data("tests/ysi_test_files", "test.csv", baytdat)

False

In [20]:
df.to_csv("tests/ysi_test_files/filehere.csv", index = False)

In [21]:
root_dir = "T:/baysestuaries/PROCEDURES_AND_ANALYSES/DATASONDE/DS_Data_from_TPWD/THIS_JUST_IN/DS_DatafmParks2017_0630/0517MATA.dat"

In [22]:
metadata, DF = sonde3.sonde(root_dir)

C:\Users\ETurner\Desktop\python\sonde3\sonde3\sonde.py:19: UserWarning: Info: No time zone was set for file, assuming records are recorded in CST
  metadata, df = formats.read_ysi(filename, tzinfo)


In [23]:
DF

,Datetime_(UTC),water_temp_c,water_conductivity_mS/cm,water_depth_m_nonvented,instrument_battery_voltage,water_specific_conductivity_mS/cm,water_salinity_PSU
0,2017-05-12 11:00:08+00:00,23.019409,0.570084,-0.000000,6.328125,0.592497,0.288141
1,2017-05-12 12:00:08+00:00,22.967224,0.569611,-0.000000,6.328125,0.592620,0.288216
2,2017-05-12 13:00:08+00:00,23.015747,0.569809,-0.000000,6.250000,0.592255,0.288021
3,2017-05-12 14:00:08+00:00,23.089905,0.569962,-0.000000,6.328125,0.591543,0.287646
4,2017-05-12 15:00:08+00:00,25.259705,46.257812,2.484802,6.250000,46.029490,29.828772
5,2017-05-12 16:00:08+00:00,25.293579,46.066406,2.415710,6.250000,45.809536,29.668860
6,2017-05-12 17:00:08+00:00,25.397034,46.148438,2.396362,6.328125,45.801112,29.660463
7,2017-05-12 18:00:08+00:00,25.558167,46.445312,2.238098,6.250000,45.955383,29.768477
8,2017-05-12 19:00:08+00:00,25.834656,46.787109,2.373108,6.250000,46.052937,29.832635
9,2017-05-12 20:00:08+00:00,26.014099,46.302734,2.364136,6.250000,45.422924,29.372858
